<a href="https://colab.research.google.com/github/Muhammadamin0334/JEPA/blob/main/Jepa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install latest transformers (with VJEPA2 support)
!pip install git+https://github.com/huggingface/transformers.git

# Install additional required packages
!pip install opencv-python-headless fastrtc gradio loguru

# Install GPU-enabled PyTorch (for CUDA 11.8 — adjust if needed)
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-fi4ae9de
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-fi4ae9de
  Resolved https://github.com/huggingface/transformers.git to commit 89b35be618256d2a4a2458322a0653c57e8fa986
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.53.0.dev0-py3-none-any.whl size=11465002 sha256=a9c6163da5b76e4bb3bffd63bbf540b7c54b601ce1a90e2b0179962087b0168f
  Stored in directory: /tmp/pip-ephem-wheel-cache-8kxyy5bd/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.52.4
    Uninstalling transformers-4.52.4:
      Successfully uninstalled transformers-4.52.4
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import gradio as gr
import cv2
import numpy as np
import torch
import time
from transformers import VJEPA2ForVideoClassification, AutoVideoProcessor

# Load model
checkpoint = "facebook/vjepa2-vitl-fpc16-256-ssv2"
device = "cuda" if torch.cuda.is_available() else "cpu"
model = VJEPA2ForVideoClassification.from_pretrained(checkpoint, torch_dtype=torch.float16).to(device)
processor = AutoVideoProcessor.from_pretrained(checkpoint)
frames_per_clip = model.config.frames_per_clip

def predict_from_video(video_file):
    if video_file is None:
        return "No video uploaded."

    cap = cv2.VideoCapture(video_file)
    frames = []
    success, frame = cap.read()
    while success:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame)
        success, frame = cap.read()
    cap.release()

    if len(frames) < frames_per_clip:
        return "Video too short."

    step = len(frames) // frames_per_clip
    selected_frames = frames[::step][:frames_per_clip]
    inputs = processor(selected_frames, return_tensors="pt").to(device, torch.float16)

    with torch.no_grad():
        logits = model(**inputs).logits
    pred = model.config.id2label[logits.argmax(-1).item()]
    return f"Prediction: {pred}"

gr.Interface(
    fn=predict_from_video,
    inputs=gr.Video(label="Upload a short video"),  # Removed `type="filepath"`
    outputs=gr.Text(label="Action Prediction"),
    title="VJEPA2 Video Classification"
).launch()


config.json:   0%|          | 0.00/15.0k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.50G [00:00<?, ?B/s]